In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
types = {i: 'bool' for i in list(range(5, 11+1)) + list(range(13,14+1)) + list(range(19, 27+1)) + list(range(29, 69+1))}
types[18] = 'str'
df=pd.read_csv("data/reports_anonymous.csv", sep=";", dtype = types, na_values = ['1/1900'], encoding = 'utf8')
#df

In [3]:
print(len(df.columns))

72


In [4]:
print(df.columns)

Index(['ETA_PRLIEVO', 'DATA_PRELIEVO', 'DATA_ULTIMA_MESTRUAZIONE',
       'DATA_PAP_PRECEDENTE', 'ESITO_PAP_PRECEDENTE', 'FORNICE', 'PORTIO',
       'CAN_CERV', 'CAV_UT', 'PERDITE_ER', 'LEUCORREA', 'PRURITO',
       'DATI_OBIETTIVI', 'TER_RAD', 'TER_ORM', 'TIPO_TER_ORM', 'TIPO_OG',
       'GRAVIDANZE', 'ABORTI', 'CTRL_3', 'CTRL_6', 'CTRL_12', 'TEST_ESTR',
       'TER_ANTIF', 'CTRL_BIOPSIA_CERV', 'CTRL_RASC_ENDOCERVICALE',
       'CTRL_RASC_ENDOMETRIALE', 'CTRL_ALTRO', 'NOTE', 'Z31_1', 'Z31_2',
       'Z31_3', 'Z32_1', 'Z32_2', 'Z32_3', 'Z32_4', 'Z32_5', 'Z32_6', 'Z32_7',
       'Z32_8', 'Z34_1', 'Z35_1', 'Z35_2', 'Z35_3', 'Z35_4', 'Z35_5', 'Z38_1',
       'Z38_2', 'Z38_3', 'Z38_4', 'Z38_5', 'Z38_6', 'Z40_1', 'Z41_1', 'Z41_2',
       'Z43_1', 'Z43_2', 'Z43_3', 'Z43_4', 'Z44_1', 'Z45_1', 'Z46_1', 'Z46_2',
       'Z46_3', 'Z47_1', 'Z47_2', 'Z47_3', 'Z48_1', 'Z48_2', 'Z48_3', 'OP_GIN',
       'Unnamed: 71'],
      dtype='object')


In [5]:
print(len(df.index))

9323


In [6]:
# drop cols 'Unnamed 71'
df.drop(['Unnamed: 71'], axis=1, inplace=True)
# drop cols 'DATI_OBIETTIVI'
df.drop(['DATI_OBIETTIVI'], axis=1, inplace=True)
# drop cols 'TIPO_OG'
df.drop(['TIPO_OG'], axis=1, inplace=True)
# drop cols 'TIPO_TER_ORM'
df.drop(['TIPO_TER_ORM'], axis=1, inplace=True)

# rename 'ETA_PRLIEVO' to 'ETA_PRELIEVO'
df.rename(index=str, columns={'ETA_PRLIEVO': 'ETA_PRELIEVO'}, inplace = True)

#df

In [7]:
# clean dates

# insert new columns to categorize some statuses that were present in the 'DATA_ULTIMA_MESTRUAZIONE'
# column (mixed with the dates)
df.loc[df['DATA_ULTIMA_MESTRUAZIONE'] == 'MENOPAUSA', 'MENOPAUSA'] = True 
df.loc[df['DATA_ULTIMA_MESTRUAZIONE'] == 'AMENORREA', 'AMENORREA'] = True 
df.loc[df['DATA_ULTIMA_MESTRUAZIONE'] == 'POST_PARTUM', 'POST_PARTUM'] = True 

# replace NaNs with False in the new columns
df[['MENOPAUSA', 'AMENORREA', 'POST_PARTUM']] = df[['MENOPAUSA', 'AMENORREA', 'POST_PARTUM']] \
                                                .fillna(value = False)
# replace previous values (and some outliar - 8/2621 that has been found while casting to datetime, in the next cell)
df['DATA_ULTIMA_MESTRUAZIONE'].replace(to_replace = ['8/2621', 'MENOPAUSA', 'AMENORREA', 'POST_PARTUM'],\
                                       value = np.NaN, \
                                       inplace = True)

#df

In [8]:
# drop rows that have 'DATA_PRELIEVO' and 'DATA_ULTIMA_MESTRUAZIONE' to NaN and 'ETA_PRELIEVO' < 12 or = NaN

df.drop(df[(df['ETA_PRELIEVO'] < 12) | (df['ETA_PRELIEVO'].isnull())].index , inplace = True) 
df.drop(df[(df['DATA_PRELIEVO'].isnull()) & (df['DATA_ULTIMA_MESTRUAZIONE'].isnull())].index, inplace = True)

print(len(df[df['ETA_PRELIEVO'] < 12 | (df['ETA_PRELIEVO'].isnull())]))
print(len(df[(df['DATA_PRELIEVO'].isnull()) & (df['DATA_ULTIMA_MESTRUAZIONE'].isnull())]))

0
0


In [9]:
# cast 'DATA_PRELIEVO' and 'DATA_ULTIMA_MESTRUAZIONE' to datetime
df['DATA_PRELIEVO'] = pd.to_datetime(df['DATA_PRELIEVO'], format = '%m/%Y')
df['DATA_ULTIMA_MESTRUAZIONE'] = pd.to_datetime(df['DATA_ULTIMA_MESTRUAZIONE'], format = '%m/%Y')


#df

In [10]:
# Drop rows that have 'DATA_PRELIEVO' < 'DATA_ULTIMA_MESTRUAZIONE'
print(len(df[df['DATA_PRELIEVO'] < df['DATA_ULTIMA_MESTRUAZIONE']]))
df.drop(df[(df['DATA_PRELIEVO'] < df['DATA_ULTIMA_MESTRUAZIONE'])].index , inplace = True) 
print(len(df[df['DATA_PRELIEVO'] < df['DATA_ULTIMA_MESTRUAZIONE']]))


49
0


In [11]:
# Cast 'ETA_PRELIEVO' to integer value

df['ETA_PRELIEVO'] = df['ETA_PRELIEVO'].astype(np.int64)
#df

In [12]:
df['DATA_PRELIEVO'].describe()

count                    8805
unique                     72
top       2014-09-01 00:00:00
freq                      220
first     1980-10-01 00:00:00
last      2018-12-01 00:00:00
Name: DATA_PRELIEVO, dtype: object

In [13]:
df['ETA_PRELIEVO'].describe()

count    8805.000000
mean       42.578421
std        12.502608
min        12.000000
25%        33.000000
50%        42.000000
75%        51.000000
max        88.000000
Name: ETA_PRELIEVO, dtype: float64

In [14]:
# Dropping outlier
df.sort_values('DATA_PRELIEVO')
df.drop(df[(df['DATA_PRELIEVO'] == '1980-10-01')].index , inplace = True) 


In [15]:
df['DATA_PRELIEVO'].describe()

count                    8804
unique                     71
top       2014-09-01 00:00:00
freq                      220
first     2007-11-01 00:00:00
last      2018-12-01 00:00:00
Name: DATA_PRELIEVO, dtype: object

In [16]:
# check the 'DATA_PAP_PRECEDENTE' column

df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].astype(str)
df = df.sort_values('DATA_PAP_PRECEDENTE')

#for n,i in enumerate(df['DATA_PAP_PRECEDENTE'].astype(str).unique()):
#    print(n,'   ', i)

In [17]:
# Replace '1°pap' through regular expression

df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'(.*PAP.*)','PRIMO_PAP', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(GE\D*)','01/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(FE\D*)','02/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(MAR\D*)','03/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(AP\D*)','04/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(MAG\D*)','05/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(GI\D*)','06/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(LU\D*)','07/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(AG\D*)','08/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(SE\D*)','09/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(OT\D*)','10/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(NO\D*)','11/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace(r'^(DI\D*)','12/', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace('01/2014.','01/2014', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace('2013 SETT.','09/2013', case = False)
df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace('2014 APRILE','04/2014', case = False)

df['DATA_PAP_PRECEDENTE'] = df['DATA_PAP_PRECEDENTE'].str.replace('06/25','06/2015', case = False)




In [18]:
# Substitution and computation of the 'DATA_PAP_PRECEDENTE' in case of the value is 'XX aa fa'
for index, row in df[['DATA_PRELIEVO', 'DATA_PAP_PRECEDENTE']].astype(str).iterrows():
    result = re.findall(r'^(\d{1,2})\s*AA', row['DATA_PAP_PRECEDENTE'], re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'DATA_PAP_PRECEDENTE'] = df.loc[index, 'DATA_PRELIEVO'] - pd.Timedelta(days=int(result[0])*365)
    

In [19]:
# Substitution and computation of the 'DATA_PAP_PRECEDENTE' in case of the value is 'XX mesi fa'
for index, row in df[['DATA_PRELIEVO', 'DATA_PAP_PRECEDENTE']].astype(str).iterrows():
    result = re.findall(r'^(\d{1,2})\s*MESI', row['DATA_PAP_PRECEDENTE'], re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'DATA_PAP_PRECEDENTE'] = df.loc[index, 'DATA_PRELIEVO'] - pd.Timedelta(days=int(result[0])*30)
    

In [20]:
# Substitution and computation of the 'DATA_PAP_PRECEDENTE' in case of the value is 'xx/xx'
for index, item in df['DATA_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'^(\d{1,2})\/(\d{2})$', item)
    if len(result) == 0 or len(result[0]) < 2:
        continue
    df.loc[index, 'DATA_PAP_PRECEDENTE'] = str(result[0][0])+'/'+'20'+str(result[0][1])
    

In [21]:
#df[df['DATA_PAP_PRECEDENTE'] == 'PRIMO_PAP']

In [22]:
# insert new columns to categorize  'PRIMO_PAP'
df.loc[df['DATA_PAP_PRECEDENTE'] == 'PRIMO_PAP', 'PRIMO_PAP'] = True 

# replace NaNs with False in the new columns
df['PRIMO_PAP'] = df['PRIMO_PAP'].fillna(value = False)
# replace previous values 
df['DATA_PAP_PRECEDENTE'].replace(to_replace = ['PRIMO_PAP'], value = np.NaN, inplace = True)

len(df[df['PRIMO_PAP'] == True])

87

In [23]:
# Cast of all value in 'DATA_PAP_PRECEDENTE' to datetime
df['DATA_PAP_PRECEDENTE'] = pd.to_datetime(df['DATA_PAP_PRECEDENTE'])

#df

In [24]:
# Every row that has not a value in 'DATA_PAP_PRECEDENTE' and 'ESITO_PAP_PRECEDENTE' are considered to be 'PRIMO_PAP'
df.loc[(df['DATA_PAP_PRECEDENTE'].isnull()) & (df['ESITO_PAP_PRECEDENTE'].isnull()), 'PRIMO_PAP'] = True 
len(df[df['PRIMO_PAP'] == True])

6570

In [25]:
# check date columns
df.columns

Index(['ETA_PRELIEVO', 'DATA_PRELIEVO', 'DATA_ULTIMA_MESTRUAZIONE',
       'DATA_PAP_PRECEDENTE', 'ESITO_PAP_PRECEDENTE', 'FORNICE', 'PORTIO',
       'CAN_CERV', 'CAV_UT', 'PERDITE_ER', 'LEUCORREA', 'PRURITO', 'TER_RAD',
       'TER_ORM', 'GRAVIDANZE', 'ABORTI', 'CTRL_3', 'CTRL_6', 'CTRL_12',
       'TEST_ESTR', 'TER_ANTIF', 'CTRL_BIOPSIA_CERV',
       'CTRL_RASC_ENDOCERVICALE', 'CTRL_RASC_ENDOMETRIALE', 'CTRL_ALTRO',
       'NOTE', 'Z31_1', 'Z31_2', 'Z31_3', 'Z32_1', 'Z32_2', 'Z32_3', 'Z32_4',
       'Z32_5', 'Z32_6', 'Z32_7', 'Z32_8', 'Z34_1', 'Z35_1', 'Z35_2', 'Z35_3',
       'Z35_4', 'Z35_5', 'Z38_1', 'Z38_2', 'Z38_3', 'Z38_4', 'Z38_5', 'Z38_6',
       'Z40_1', 'Z41_1', 'Z41_2', 'Z43_1', 'Z43_2', 'Z43_3', 'Z43_4', 'Z44_1',
       'Z45_1', 'Z46_1', 'Z46_2', 'Z46_3', 'Z47_1', 'Z47_2', 'Z47_3', 'Z48_1',
       'Z48_2', 'Z48_3', 'OP_GIN', 'MENOPAUSA', 'AMENORREA', 'POST_PARTUM',
       'PRIMO_PAP'],
      dtype='object')

In [26]:
df['DATA_PRELIEVO'].describe()

count                    8804
unique                     71
top       2014-09-01 00:00:00
freq                      220
first     2007-11-01 00:00:00
last      2018-12-01 00:00:00
Name: DATA_PRELIEVO, dtype: object

In [27]:
print(len(df[df['DATA_PRELIEVO'] > '2018-06-01']))
df.loc[(df['DATA_PRELIEVO'] > '2018-06-01', 'DATA_PRELIEVO')] = np.datetime64('NaT')
print(len(df[df['DATA_PRELIEVO'] > '2018-06-01']))

2
0


In [28]:
df['DATA_PAP_PRECEDENTE'].describe()

count                    2178
unique                     81
top       2012-01-01 00:00:00
freq                      415
first     1993-01-01 00:00:00
last      2016-10-01 00:00:00
Name: DATA_PAP_PRECEDENTE, dtype: object

In [29]:
df['DATA_ULTIMA_MESTRUAZIONE'].describe()

count                    5314
unique                     96
top       1905-07-01 00:00:00
freq                      162
first     1905-06-01 00:00:00
last      2018-03-01 00:00:00
Name: DATA_ULTIMA_MESTRUAZIONE, dtype: object

In [30]:
print(len(df[df['DATA_ULTIMA_MESTRUAZIONE'] < '1985-01-01']))
df.loc[(df['DATA_ULTIMA_MESTRUAZIONE']< '1985-01-01', 'DATA_ULTIMA_MESTRUAZIONE')] = np.datetime64('NaT')
print(len(df[df['DATA_ULTIMA_MESTRUAZIONE'] < '1985-01-01']))

239
0


In [31]:
# check the 'ESITO_PAP_PRECEDENTE' column

df['ESITO_PAP_PRECEDENTE'] = df['ESITO_PAP_PRECEDENTE'].astype(str)
df = df.sort_values('ESITO_PAP_PRECEDENTE')

#for n,i in enumerate(df['ESITO_PAP_PRECEDENTE'].astype(str).unique()):
#    print(n,'   ', i)

In [32]:
# Refactoring '1°PAP TEST/1 PAP TEST' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'1°PAP TEST|1 PAP TEST', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'PRIMO_PAP'] = True
    
print(len(df[df['PRIMO_PAP']== True]))

6572


In [33]:
# women in which 'DATA_PRELIEVO' - 'DATA_ULTIMA_MESTRUAZIONE' > 2 years are considered to be in 'MENOPAUSA'

print(len(df[df['MENOPAUSA'] == True]))
print(len(df[df['DATA_PRELIEVO'] - df['DATA_ULTIMA_MESTRUAZIONE'] >= pd.Timedelta(days=2*365)]))

df.loc[(df['DATA_PRELIEVO'] - df['DATA_ULTIMA_MESTRUAZIONE'] >= pd.Timedelta(days=2*365)), 'MENOPAUSA'] = True

print(len(df[df['MENOPAUSA'] == True]))

1085
31
1116


In [34]:
### CITO ###

In [35]:
# Refactoring 'NEGATIVE' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'(n|m)eg|infiammazione|ipercheratosi|metaplasia|endocervicale|squame|atrofico|citolisi|cervicite',\
                        item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'CITO_PREC'] = 'NEGATIVO'
    
print(len(df[df['CITO_PREC'] == 'NEGATIVO']))

1911


In [36]:
# Refactoring 'ASCUS' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'AGUS|ACSUS|ASCUS', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'CITO_PREC'] = 'ASCUS'
    
print(len(df[df['CITO_PREC']== 'ASCUS']))

38


In [37]:
# Refactoring 'LSIL/CIN1' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'l\ ?-?\ ?sil| cin\ ?1|BASSO\ RISCHIO|CONDILOMNA|DISPLASIA', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'CITO_PREC'] = 'LSIL'
    
print(len(df[df['CITO_PREC']== 'LSIL']))

68


In [38]:
# Refactoring 'HSIL/CIN2/CIN3' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'h\ ?-?\ ?sil| cin\ ?(2|3)|LIEVE E MODERATA', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'CITO_PREC'] = 'HSIL'
    
print(len(df[df['CITO_PREC']== 'HSIL']))

16


In [39]:
# Refactoring 'ASC-H' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'ASC-H', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'CITO_PREC'] = 'ASC-H'
    
print(len(df[df['CITO_PREC']== 'ASC-H']))

1


In [40]:
# Refactoring 'CARCINOMA IN SITU' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'carcinoma in situ', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'CITO_PREC'] = 'SCC'
    
print(len(df[df['CITO_PREC']== 'SCC']))

1


In [41]:
# Refactoring 'AGC' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'AGC', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'CITO_PREC'] = 'AGC-NOS'
    
print(len(df[df['CITO_PREC']== 'AGC-NOS']))

1


In [42]:
### VIRAL ###

In [43]:
# Refactoring 'HPV' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'HPV|P\d+|16', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'VIRALE_PREC'] = 'HPV'
    
print(len(df[df['VIRALE_PREC']== 'HPV']))

136


In [44]:
# Refactoring 'HIV' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'HIV', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'VIRALE_PREC'] = 'HIV'
    
print(len(df[df['VIRALE_PREC']== 'HIV']))

1


In [45]:
# Refactoring 'HERPES' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'HERPES', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'VIRALE_PREC'] = 'HERPES'
    
print(len(df[df['VIRALE_PREC']== 'HERPES']))

1


In [46]:
### BATT ###

In [47]:
# Refactoring 'VAGINOSI' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'VAGINOSI', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT_PREC'] = 'VAGINOSI'
    
print(len(df[df['BATT_PREC']== 'VAGINOSI']))

2


In [48]:
# Refactoring 'FLOGOSI' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'FLOGOSI', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT_PREC'] = 'FLOGOSI'
    
print(len(df[df['BATT_PREC']== 'FLOGOSI']))

2


In [49]:
# Refactoring 'ACTINOMYCES' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'ACTINOMYCES', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT_PREC'] = 'ACTINOMYCES'
    
print(len(df[df['BATT_PREC']== 'ACTINOMYCES']))

1


In [50]:
# Refactoring 'TRICHOMONAS' values in the 'ESITO_PAP_PRECEDENTE' column
for index, item in df['ESITO_PAP_PRECEDENTE'].astype(str).iteritems():
    result = re.findall(r'TRICHOMONAS', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT_PREC'] = 'TRICHOMONAS'
    
print(len(df[df['BATT_PREC']== 'TRICHOMONAS']))

1


In [51]:
len(df[df['CITO_PREC']=='NEGATIVO'])

1899

In [52]:
#df

In [53]:
# drop 'ESITO_PAP_PRECEDENTE' column
df.drop(['ESITO_PAP_PRECEDENTE'], axis=1, inplace=True)
#df

In [54]:
#for i in df.columns:
#    print(i)


#df.columns

In [55]:
# check the 'GRAVIDANZE' column

df['GRAVIDANZE'] = df['GRAVIDANZE'].astype(str)
df = df.sort_values('GRAVIDANZE')

#for n,i in enumerate(df['GRAVIDANZE'].astype(str).unique()):
#    print(n,'   ', i)

In [56]:
df['GRAVIDANZE'] = df['GRAVIDANZE'].str.replace('11','1', case = False)
df['GRAVIDANZE'] = df['GRAVIDANZE'].str.replace('1EXTRA','1', case = False)
df['GRAVIDANZE'] = df['GRAVIDANZE'].str.replace('22','2', case = False)
df['GRAVIDANZE'] = df['GRAVIDANZE'].str.replace('Q2','2', case = False)

In [57]:
# check the 'GRAVIDANZE' column

df['GRAVIDANZE'] = df['GRAVIDANZE'].astype(str)
df = df.sort_values('GRAVIDANZE')

df['GRAVIDANZE'].replace(to_replace=['NaN', 'nan'], value='0',inplace=True)

#for n,i in enumerate(df['GRAVIDANZE'].astype(str).unique()):
#    print(n,'   ', i)

In [58]:
# check the 'ABORTI' column

df['ABORTI'] = df['ABORTI'].astype(str)
df = df.sort_values('ABORTI')

df['ABORTI'].replace(to_replace=['NaN', 'nan'], value='0',inplace=True)

#for n,i in enumerate(df['ABORTI'].astype(str).unique()):
#    print(n,'   ', i)

In [59]:
df['ABORTI'] = df['ABORTI'].str.replace(r'1\(IVG\)','1', case = False)

In [60]:
# check the 'ABORTI' column

df['ABORTI'] = df['ABORTI'].astype(str)
df = df.sort_values('ABORTI')

#for n,i in enumerate(df['ABORTI'].astype(str).unique()):
#    print(n,'   ', i)

In [61]:
############
############
############

In [62]:
#for i in df.columns:
#    print(i)


#df.columns

In [63]:
# Create new nominal variable based on the values of the 'CTRL_X' boolean variables
columns_to_drop=[]
df.loc[df['CTRL_3'] == True, 'CONTROLLO'] = 3
df.loc[df['CTRL_6'] == True, 'CONTROLLO'] = 6
df.loc[df['CTRL_12'] == True, 'CONTROLLO'] = 12

columns_to_drop.extend(('CTRL_3', 'CTRL_6', 'CTRL_12'))

print(len(df[df['CONTROLLO'] == 3]))
print(len(df[df['CONTROLLO'] == 6]))
print(len(df[df['CONTROLLO'] == 12]))

2
125
8081


In [64]:
# Create new nominal variable based on the values of the 'CTRL_XXXX' boolean variables
df.loc[df['CTRL_BIOPSIA_CERV'] == True, 'CTRL_ISTOLOGICO'] = 'BIOPSIA CERVICALE'
df.loc[df['CTRL_RASC_ENDOCERVICALE'] == True, 'CTRL_ISTOLOGICO'] = 'RASCHIAMENTO ENDOCERVICALE'
df.loc[df['CTRL_RASC_ENDOMETRIALE'] == True, 'CTRL_ISTOLOGICO'] = 'RASCHIAMENTO ENDOMETRIALE'
df.loc[df['CTRL_ALTRO'] == True, 'CTRL_ISTOLOGICO'] = 'ALTRO'

columns_to_drop.extend(('CTRL_BIOPSIA_CERV', 'CTRL_RASC_ENDOCERVICALE', 'CTRL_RASC_ENDOMETRIALE', 'CTRL_ALTRO'))

print(len(df[df['CTRL_ISTOLOGICO'] == 'BIOPSIA CERVICALE']))
print(len(df[df['CTRL_ISTOLOGICO'] == 'RASCHIAMENTO ENDOCERVICALE']))
print(len(df[df['CTRL_ISTOLOGICO'] == 'RASCHIAMENTO ENDOMETRIALE']))
print(len(df[df['CTRL_ISTOLOGICO'] == 'ALTRO']))

1
3
5
0


In [65]:
# Create new nominal variable based on the values of the 'Z31_X' boolean variables
df.loc[df['Z31_1'] == True, 'VALIDITA_CAMPIONE'] = 'ADEGUATO'
df.loc[df['Z31_2'] == True, 'VALIDITA_CAMPIONE'] = 'SUBOTTIMALE'
df.loc[df['Z31_3'] == True, 'VALIDITA_CAMPIONE'] = 'INADEGUATO'

columns_to_drop.extend(('Z31_1', 'Z31_2', 'Z31_3'))

print(len(df[df['VALIDITA_CAMPIONE'] == 'ADEGUATO']))
print(len(df[df['VALIDITA_CAMPIONE'] == 'SUBOTTIMALE']))
print(len(df[df['VALIDITA_CAMPIONE'] == 'INADEGUATO']))

8625
2
18


In [66]:
# Create new nominal variable based on the values of the 'Z32_X' boolean variables
df.loc[df['Z32_1'] == True, 'CAUSA_CAMPIONE_NON_VALIDO'] = 'CELLULARITA_SCARSA'
df.loc[df['Z32_2'] == True, 'CAUSA_CAMPIONE_NON_VALIDO'] = 'CONSERVAZIONE_O_FISSAZIONE_DIFETTOSA'
df.loc[df['Z32_3'] == True, 'CAUSA_CAMPIONE_NON_VALIDO'] = 'PRESENZA_DI_MATERIALE_ESTRANEO'
df.loc[df['Z32_4'] == True, 'CAUSA_CAMPIONE_NON_VALIDO'] = 'INFIAMMAZIONE_CHE_OFFUSCA_COMPLETAMENTE_O_PARZIALMENTE'
df.loc[df['Z32_5'] == True, 'CAUSA_CAMPIONE_NON_VALIDO'] = 'SANGUE_CHE_OFFUSCA_COMPLETAMENTE_O_PARZIALMENTE'
df.loc[df['Z32_6'] == True, 'CAUSA_CAMPIONE_NON_VALIDO'] = 'CITOLISI_O_AUTOLISI_ECCESSIVA'
df.loc[df['Z32_7'] == True, 'CAUSA_CAMPIONE_NON_VALIDO'] = 'NESSUNA_COMPONENTE_ENDOCERVICALE_NO_IN_MENOPAUSA'
df.loc[df['Z32_8'] == True, 'CAUSA_CAMPIONE_NON_VALIDO'] = 'NON_RAPPRESENTATIVO_DELLA_SEDE_ANATOMICA'

columns_to_drop.extend(('Z32_1', 'Z32_2', 'Z32_3', 'Z32_4', 'Z32_5', 'Z32_6', 'Z32_7', 'Z32_8'))

print(len(df[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'CELLULARITA_SCARSA']))
print(len(df[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'CONSERVAZIONE_O_FISSAZIONE_DIFETTOSA']))
print(len(df[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'PRESENZA_DI_MATERIALE_ESTRANEO']))
print(len(df[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'INFIAMMAZIONE_CHE_OFFUSCA_COMPLETAMENTE_O_PARZIALMENTE']))
print(len(df[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'SANGUE_CHE_OFFUSCA_COMPLETAMENTE_O_PARZIALMENTE']))
print(len(df[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'CITOLISI_O_AUTOLISI_ECCESSIVA']))
print(len(df[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'NESSUNA_COMPONENTE_ENDOCERVICALE_NO_IN_MENOPAUSA']))
print(len(df[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'NON_RAPPRESENTATIVO_DELLA_SEDE_ANATOMICA']))

18
0
0
0
10
0
0
0


In [67]:
# il numero di individui che hanno 'CAUSA_CAMPIONE_NON_VALIDO' non corrisponde al numero di 'INADEGUATO'
df[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'CELLULARITA_SCARSA']

,ETA_PRELIEVO,DATA_PRELIEVO,DATA_ULTIMA_MESTRUAZIONE,DATA_PAP_PRECEDENTE,FORNICE,PORTIO,CAN_CERV,CAV_UT,PERDITE_ER,LEUCORREA,...,AMENORREA,POST_PARTUM,PRIMO_PAP,CITO_PREC,VIRALE_PREC,BATT_PREC,CONTROLLO,CTRL_ISTOLOGICO,VALIDITA_CAMPIONE,CAUSA_CAMPIONE_NON_VALIDO
8215,50,2016-03-01,NaT,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,CELLULARITA_SCARSA
2105,45,2016-03-01,2016-03-01,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,CELLULARITA_SCARSA
6045,65,2015-01-01,NaT,NaT,False,False,False,True,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA
8356,51,2016-05-01,NaT,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,CELLULARITA_SCARSA
8291,63,2016-04-01,NaT,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,CELLULARITA_SCARSA
719,66,2014-11-01,NaT,NaT,False,False,False,True,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,CELLULARITA_SCARSA
2389,34,2016-05-01,2016-04-01,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,CELLULARITA_SCARSA
3386,43,2017-07-01,NaT,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,CELLULARITA_SCARSA
3354,77,2017-06-01,NaT,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,CELLULARITA_SCARSA
1456,54,2015-06-01,NaT,NaT,False,False,False,True,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA


In [68]:
# correzione degli 'INADEGUATO' in base a 'CAUSA_CAMPIONE_NON_VALIDO'
df.loc[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'CELLULARITA_SCARSA', 'VALIDITA_CAMPIONE'] = 'INADEGUATO'
df[df['CAUSA_CAMPIONE_NON_VALIDO'] == 'CELLULARITA_SCARSA']


,ETA_PRELIEVO,DATA_PRELIEVO,DATA_ULTIMA_MESTRUAZIONE,DATA_PAP_PRECEDENTE,FORNICE,PORTIO,CAN_CERV,CAV_UT,PERDITE_ER,LEUCORREA,...,AMENORREA,POST_PARTUM,PRIMO_PAP,CITO_PREC,VIRALE_PREC,BATT_PREC,CONTROLLO,CTRL_ISTOLOGICO,VALIDITA_CAMPIONE,CAUSA_CAMPIONE_NON_VALIDO
8215,50,2016-03-01,NaT,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA
2105,45,2016-03-01,2016-03-01,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA
6045,65,2015-01-01,NaT,NaT,False,False,False,True,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA
8356,51,2016-05-01,NaT,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA
8291,63,2016-04-01,NaT,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA
719,66,2014-11-01,NaT,NaT,False,False,False,True,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA
2389,34,2016-05-01,2016-04-01,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA
3386,43,2017-07-01,NaT,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA
3354,77,2017-06-01,NaT,NaT,False,True,True,False,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA
1456,54,2015-06-01,NaT,NaT,False,False,False,True,False,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,INADEGUATO,CELLULARITA_SCARSA


In [69]:
print(len(df[df['VALIDITA_CAMPIONE'] == 'INADEGUATO']))


28


In [70]:
# Refactoring of bacteriological results
df.loc[df['Z35_1'] == True, 'BATT'] = 'CANDIDA'
df.loc[df['Z35_2'] == True, 'BATT'] = 'VAGINOSI'
df.loc[df['Z35_3'] == True, 'BATT'] = 'ACTINOMYCES'
df.loc[df['Z35_4'] == True, 'BATT'] = 'TRICHOMONAS'
df.loc[df['Z35_5'] == True, 'BATT'] = 'CLAMYDIA'



columns_to_drop.extend(('Z35_1', 'Z35_2', 'Z35_3', 'Z35_4', 'Z35_5'))

print(len(df[df['BATT'] == 'CANDIDA']))
print(len(df[df['BATT'] == 'VAGINOSI']))
print(len(df[df['BATT'] == 'ACTINOMYCES']))
print(len(df[df['BATT'] == 'TRICHOMONAS']))
print(len(df[df['BATT'] == 'CLAMYDIA']))

182
421
10
6
0


In [71]:
# Refactoring of cytophatological results
df.loc[df['Z34_1'] == True, 'CITO'] = 'NEGATIVO'

columns_to_drop.append('Z34_1')

print(len(df[df['CITO'] == 'NEGATIVO']))

8444


In [72]:
# Refactoring of cytophatological results
df.loc[df['Z38_1'] == True, 'CITO'] = 'NEGATIVO'
df.loc[df['Z38_2'] == True, 'CITO'] = 'NEGATIVO'
df.loc[df['Z38_3'] == True, 'CITO'] = 'NEGATIVO'
df.loc[df['Z38_4'] == True, 'CITO'] = 'NEGATIVO'
df.loc[df['Z38_5'] == True, 'CITO'] = 'NEGATIVO'
df.loc[df['Z38_5'] == True, 'CITO'] = 'NEGATIVO'

columns_to_drop.extend(('Z38_1', 'Z38_2', 'Z38_3', 'Z38_4', 'Z38_5', 'Z38_6'))

print(len(df[df['CITO'] == 'NEGATIVO']))

8457


In [73]:
# Refactoring of cytophatological results
df.loc[df['Z40_1'] == True, 'CITO'] = 'ASCUS'

columns_to_drop.append('Z40_1')

print(len(df[df['CITO'] == 'ASCUS']))

130


In [74]:
# Refactoring of cytophatological results
df.loc[df['Z41_1'] == True, 'CITO'] = 'LSIL'
df.loc[df['Z41_2'] == True, 'CITO'] = 'LSIL'

columns_to_drop.extend(('Z41_1', 'Z41_2'))

print(len(df[df['CITO'] == 'LSIL']))


106


In [75]:
# Refactoring of cytophatological/ viral results
df.loc[df['Z43_1'] == True, 'CITO'] = 'HSIL'
df.loc[df['Z43_2'] == True, 'CITO'] = 'HSIL'
df.loc[df['Z43_3'] == True, 'CITO'] = 'HSIL'
df.loc[df['Z43_4'] == True, 'CITO'] = 'HSIL'
df.loc[df['Z43_4'] == True, 'VIRALE'] = 'HPV'

columns_to_drop.extend(('Z43_1', 'Z43_2', 'Z43_3', 'Z43_4'))

print(len(df[df['CITO'] == 'HSIL']))
print(len(df[df['VIRALE'] == 'HPV']))


17
1


In [76]:
# Refactoring of cytophatological results
df.loc[df['Z44_1'] == True, 'CITO'] = 'AGC-NEOPLASTIC'

columns_to_drop.append('Z44_1')

print(len(df[df['CITO'] == 'AGC-NEOPLASTIC']))

2


In [77]:
# Refactoring of cytophatological results
df.loc[df['Z45_1'] == True, 'CITO'] = 'SCC'

columns_to_drop.extend(('Z45_1', 'Z46_1', 'Z46_2', 'Z46_3' ))

print(len(df[df['CITO'] == 'SCC']))

1


In [78]:
# Refactoring of cytophatological results
df.loc[df['Z47_1'] == True, 'CITO'] = 'AGC-NOS'
df.loc[df['Z47_2'] == True, 'CITO'] = 'AGC-NOS'
df.loc[df['Z47_3'] == True, 'CITO'] = 'AGC-NOS'

columns_to_drop.extend(('Z47_1', 'Z47_2', 'Z47_3'))

print(len(df[df['CITO'] == 'AGC-NOS']))

6


In [79]:
# Refactoring of cytophatological results
df.loc[df['Z48_1'] == True, 'CITO'] = 'AIS'
df.loc[df['Z48_2'] == True, 'CITO'] = 'AIS'
df.loc[df['Z48_3'] == True, 'CITO'] = 'AIS'


columns_to_drop.extend(('Z48_1', 'Z48_2', 'Z48_3'))

print(len(df[df['CITO'] == 'AIS']))

1


In [80]:
# check the 'NOTE' column

df['NOTE'] = df['NOTE'].astype(str)
df = df.sort_values('NOTE')

for n,i in enumerate(df['NOTE'].astype(str).unique()):
    print(n,'   ', i)



0      AGC: SI CONSIGLIANO ULTERIORI ACCERTAMENTI DIAGNOSTICI
1      ALTERAZIONI CELLULARI COMPATIBILI CON INFEZIONE DA CANDIDA.SI CONSIGLIA CONTROLLO DOPO TERAPIA.
2      CELLULE SQUAMOSE ATIPICHE:NON SI ESCLUDE UNA LESIONE INTRAEPITELIALE DI BASSO GRADO(LSIL-HPV).SI CONSIGLIA COLPOSCOPIA
3     A.G.C. ALCUNE CELLULE GHIANDOLARI ATIPICHE VEROSIMILMENTE REATTIVE. INTENSO RICHIAMAO LEUCOCITARIO
4     ABBONDANTE CITOLISI BATTERICA DA DODERLEIN
5     ABBONDANTE FLORA BATTERICA DI TIPO COCCICO
6     ABBONDANTE FLORA BATTERICA DI TIPO COCCIFORME
7     ABBONDANTE FLORA BATTERICA DI TIPO COCCIFORME.NUMEROSI LEUCOCITI
8     ABBONDANTE FLORA BETTRICA DI TIPO COCCIFORME
9     ABBONDANTE FLORA DI COCCHI
10     ABBONDANTE FLORA DI COCCHI CON MODIFICAZINE DELLA NORMALE FLORA VAGINALE
11     ABBONDANTE FLORA DI COCCHI CON MODIFICAZIONE DELLA NORMALE FLORA VAGINALE
12     ABBONDANTE FLORA DI COCCHI,DISCRETI LEUCOCITI
13     ABBONDANTE FLORA DI COCCHI,DISCRETI LEUCOCITI,ALTERAZIONI CELLULARI FLOGISTICH

129     ASSENZA DI CELLULE ENDOCERVICALI
130     ASSENZA DI CELLULE ENDOCERVICALI. DODERLEIN
131     ASSENZA DI CELLULE ENDOCERVICALI.DODERLEIN
132     ASSENZA DI CELLULE ENDOCERVICALI.MICETI MORFOLOGICAMENTE INDICATIVI DI CANDIDA spp
133     ASSENZA DI CELLULE ENDOCERVIVALI
134     ATOFIA EPITELIALE
135     ATR0FIA EPITELIALE
136     ATROFIA
137     ATROFIA A CELLULE INTERMEDIE
138     ATROFIA A CELLULE INTERMEDIE.ASSENTE LA COMPONENTE ENDOCERVICALE
139     ATROFIA A CELLULE INTERMEDIE.DISCRETO RICHIAMO LEUCOCITARIO
140     ATROFIA EPITELIALE
141     ATROFIA EPITELIALE.DIVERSI LEUCOCITI
142     ATROFIA EPITELIALE.FLORA BATTERICA MISTA
143     ATROFIA EPITELIALE.ISTIOCITI GIGANTI
144     ATROFIA EPITELUIALE
145     ATROFIA EPTELIALE
146     BACILLO DI DODERLEIN
147     BACILLO DI DODERLEIN.
148     BATTERI MORFOLOGICAMENTE COMPATIBILI CON ACTINOMYCES
149     CARCINOMO SQUAMOSONON CHERATINIZZANTE
150     CELLELE DA LESIONE SQUAMOSA INTRAEPITELIALE DI BASSO GRADO.SI CONSIGLIA COLOSCOPIA


313     CONTROLLLO A 3/4 MESI DOPO TERAPIA
314     CONTROLLO A  3/4 MESI DOPO TERAPIA
315     CONTROLLO A 3/4 MESI DOPO TERAPIA
316     CONTROLLO DOPO TERAPIA
317     DI9SCRETO RICHIAMO LEUCOCITARIO
318     DICRETO RICHIAMO LEUCOCITARIO
319     DICRETO RICHIAMO LEUCOCITARIO, NUMEROSE SPORE FUNGINE.
320     DIFFUSA IPERCHERATOSI -PARACHERATOSI.SI CONSIGLIA COLPOSCOPIA
321     DIFFUSA IPERCHERATOSI,FLOGOSI E SANGUE IN PARTE OSCURANTE.SI CONSIGLIA COLPOSCOPIA
322     DIFFUSA IPERCHERATOSI-METAPLASIA SQUAMOSA.SI CONSIGLIA COLPOSCOPIA
323     DIFFUSA IPERCHERATOSI-PARACHERATOSI.SI CONSIGLIANO ULTERIORI ACCERTAMENTI DIAGNOSTICI
324     DIFFUSA IPERCHERATOSI.CITOLISI BATTERICA
325     DIFFUSA IPERCHERATOSI.SI CONSIGLIA COLPOSCOPIA
326     DIFFUSA IPERCHERATOSI.SI CONSIGLIANO ULTERIORI ACCERTAMENTI DIAGNOSTICI
327     DIFFUSA IPERCHERATOSI.VAGINOSI BATTERICA
328     DIFFUSA METAPLASIA SQUAMOSA
329     DIFFUSA METAPLASIA SQUAMOSA IMMATURA
330     DIFFUSA METAPLASIA SQUAMOSA MATURA ED IMMATURA.


480     INTENSA CITOLISI BATTERICA
481     INTENSA CITOLISI BATTERICA DA DODERLEIN
482     INTENSA CITOLISI BATTERICA DA DODERLEIN CON NUCLEI NUDI
483     INTENSA CITOLISI BATTERICA DA DODERLEIN CON NUCLEI NUDI.ALTERAZIONI CELLULARI FLOGISTICHE
484     INTENSA CITOLISI BATTERICA DA DODERLEIN CON NUCLEI NUDI.SI CONSIGLIA RIPETIZIONE DOPO TERAPIA
485     INTENSA CITOLISI BATTERICA DA DODERLEIN IN PARTE OSCURANTE.SI CONSIGLIA RIPETIZIONE DOPO TERAPIA
486     INTENSA CITOLISI BATTERICA DA DODERLEIN.DISCRETI LEUCOCITI.ALTERAZIONI FLOGISTICHE
487     INTENSA FLOGOSI IN PARTE OSCURANTE LA COMPONENTE EPITELIALE SI CONSIGLIA CONTROLLO A 3/4 MESI DOPO TERAPIA
488     INTENSA FLOGOSI IN PARTE OSCURANTE. CELLULE DA RIPARO E METAPLASTICHE IMMATURE
489     INTENSA FLOGOSI IN PARTE OSCURANTE. CONTROLLO A 3/4 MESI DOPO TERAPIA
490     INTENSA FLOGOSI IN PARTE OSCURANTE.SI CONSIGLIA CONTROLLO A 3/4 MESI DOPO TERAPIA
491     INTENSA FLOGOSI. EPITELIO TROFICO
492     INTENSA FLOGOSI.ALTERAZIONI CELLULARI

646     NUMEROSE SPORE DI MICETI
647     NUMEROSE SPORE FUNGINE
648     NUMEROSE SPORE MICOTICHE
649     NUMEROSE SPOREMICOTICHE
650     NUMEROSE SQUAME CORNEE
651     NUMEROSE SQUAME CORNEE. ASSENZA DI CELLULE ENDOCERVICALI
652     NUMEROSE SQUAME CORNEEE:SI CONSIGLIA COLPOSCOPIA
653     NUMEROSI CLUSTERS GHIANDOLARI ENDOCERVICALI
654     NUMEROSI ISTIOCITI
655     NUMEROSI LEUCOCI METAPLASIA SQUAMOSA
656     PARACHERATOSI
657     PARACHERATOSI,METAPLASIA SQUAMOSA MATURA ED IMMATURA.SI CONSIGLIA COLPOSCOPIA
658     PARACHERATOSI,SI CONSIGLIA RIPETIZIONE DOPO TERAPIA
659     PARACHERATOSI. SI CONSIGLIA COLPOSCOPIA
660     PARACHERATOSI.ALTERAZIONI CELLULERI COMPATIBILI CON INFEZIONE DA CANDIDA spp
661     PARACHERATOSI.CITOLISI BATTERICA
662     PARACHERATOSI.CITOLISI BATTERICASI CONSIGLIA COLPOSCOPIA
663     PARACHERATOSI.CONTROLLO A 3/4 MESI DOPO TERAPIA
664     PARACHERATOSI.FLORA BATTERICA MISTA
665     PARACHERATOSI.METAPLASIA SQ1UAMOSA
666     PARACHERATOSI.NOTE DI METAPLASIA
667

813     VAGINOSI BATTERICA.CONTROLLO DOPO TERAPIA
814     VAGINOSI BATTERICA.IPERCHERATOSI
815     VAGINOSI BATTERICA.NOTE DI METAPLASIA SQUAMOSA.SI CONSIGLIA CONTROLLO DOPO TERAPIA
816     VAGINOSI.NOTE DI METAPLASIA
817     VETRINO ESO ED ENDO:PREVALENZA DI COCCHI CON MODIFICAZIONE DELLA NORMALE FLORA VAGINALE
818     VETRINO TRADIZIONALE:SANGUE CHE OFFUSCA COMPLETAMENTE.
VETRINO VULVA:NEGATIVO-IPERCHERATOSI
819     nan


In [81]:
# Using 'NOTE' values to fill extra values in other columns
for index, item in df['NOTE'].astype(str).iteritems():
    result = re.findall(r'DODERLEIN', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT'] = 'DODERLEIN'
    
print(len(df[df['BATT']== 'DODERLEIN']))

230


In [82]:
# Using 'NOTE' values to fill extra values in other columns
for index, item in df['NOTE'].astype(str).iteritems():
    result = re.findall(r'cocchi|coccico|cocciforme', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT'] = 'COCCHI'
    
print(len(df[df['BATT']== 'COCCHI']))

325


In [83]:
# Using 'NOTE' values to fill extra values in other columns
for index, item in df['NOTE'].astype(str).iteritems():
    result = re.findall(r'GARDNERELLA', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT'] = 'GARDNERELLA'
    
print(len(df[df['BATT']== 'GARDNERELLA']))

9


In [84]:
# Using 'NOTE' values to fill extra values in other columns
for index, item in df['NOTE'].astype(str).iteritems():
    result = re.findall(r'CANDIDA', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT'] = 'CANDIDA'
    
print(len(df[df['BATT']== 'CANDIDA']))

233


In [85]:
# Using 'NOTE' values to fill extra values in other columns
for index, item in df['NOTE'].astype(str).iteritems():
    result = re.findall(r'FLOGOSI|FLOGISTICA', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT'] = 'FLOGOSI'
    
print(len(df[df['BATT']== 'FLOGOSI']))

85


In [86]:
# Using 'NOTE' values to fill extra values in other columns
for index, item in df['NOTE'].astype(str).iteritems():
    result = re.findall(r'CLAMYDIA', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT'] = 'CLAMYDIA'
    
print(len(df[df['BATT']== 'CLAMYDIA']))

1


In [87]:
# Using 'NOTE' values to fill extra values in other columns
for index, item in df['NOTE'].astype(str).iteritems():
    result = re.findall(r'LATTOBACILLARE|LATTOBACILLI', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT'] = 'VAGINOSI'
    
print(len(df[df['BATT']== 'VAGINOSI']))

119


In [88]:
# Using 'NOTE' values to fill extra values in other columns
for index, item in df['NOTE'].astype(str).iteritems():
    result = re.findall(r'ACTINOMYCES', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT'] = 'ACTINOMYCES'
    
print(len(df[df['BATT']== 'ACTINOMYCES']))

13


In [89]:
# Using 'NOTE' values to fill extra values in other columns
for index, item in df['NOTE'].astype(str).iteritems():
    result = re.findall(r'TRICHOMONAS', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'BATT'] = 'TRICHOMONAS'
    
print(len(df[df['BATT']== 'TRICHOMONAS']))

6


In [90]:
# Using 'NOTE' values to fill extra values in other columns
for index, item in df['NOTE'].astype(str).iteritems():
    result = re.findall(r'ASC-H', item, re.IGNORECASE)
    if len(result) == 0:
        continue
    df.loc[index, 'CITO'] = 'ASC-H'
    
print(len(df[df['CITO']== 'ASC-H']))

2


In [91]:
columns_to_drop.append('NOTE')

In [92]:
df.drop(columns_to_drop, axis=1, inplace=True)

In [93]:
column_list = [
    'ETA_PRELIEVO',
    'DATA_PRELIEVO',
    'DATA_ULTIMA_MESTRUAZIONE',
    'MENOPAUSA',
    'AMENORREA',
    'POST_PARTUM',
    'PRIMO_PAP',
    'FORNICE',
    'PORTIO',
    'CAN_CERV',
    'CAV_UT',
    'PERDITE_ER',
    'LEUCORREA',
    'PRURITO',
    'TER_RAD',
    'TER_ORM',
    'GRAVIDANZE',
    'ABORTI',
    'OP_GIN',
    'DATA_PAP_PRECEDENTE',
    'CITO_PREC',
    'VIRALE_PREC',
    'BATT_PREC',
    'CONTROLLO',
    'TEST_ESTR',
    'TER_ANTIF',
    'CTRL_ISTOLOGICO',
    'VALIDITA_CAMPIONE',
    'CAUSA_CAMPIONE_NON_VALIDO',
    'BATT',
    'CITO',
    'VIRALE'
]
df.reindex(column_list, axis=1)

,ETA_PRELIEVO,DATA_PRELIEVO,DATA_ULTIMA_MESTRUAZIONE,MENOPAUSA,AMENORREA,POST_PARTUM,PRIMO_PAP,FORNICE,PORTIO,CAN_CERV,...,BATT_PREC,CONTROLLO,TEST_ESTR,TER_ANTIF,CTRL_ISTOLOGICO,VALIDITA_CAMPIONE,CAUSA_CAMPIONE_NON_VALIDO,BATT,CITO,VIRALE
4306,54,2018-04-01,NaT,False,False,False,True,False,True,True,...,NaN,NaN,False,False,NaN,ADEGUATO,NaN,NaN,AGC-NOS,NaN
7808,49,2015-11-01,2015-10-01,False,False,False,True,False,True,True,...,NaN,NaN,False,False,NaN,ADEGUATO,NaN,CANDIDA,NEGATIVO,NaN
1487,45,2015-07-01,2015-06-01,False,False,False,True,False,True,True,...,NaN,NaN,False,False,NaN,ADEGUATO,NaN,NaN,ASCUS,NaN
2132,66,2016-04-01,NaT,True,False,False,False,False,True,True,...,NaN,NaN,False,False,NaN,ADEGUATO,NaN,NaN,AGC-NOS,NaN
2016,41,2016-03-01,2016-02-01,False,False,False,False,False,True,True,...,NaN,12.0,False,False,NaN,ADEGUATO,NaN,DODERLEIN,NEGATIVO,NaN
2227,24,2016-05-01,2016-04-01,False,False,False,True,False,True,True,...,NaN,12.0,False,False,NaN,ADEGUATO,NaN,COCCHI,NEGATIVO,NaN
2340,29,2016-06-01,2016-05-01,False,False,False,True,False,True,True,...,NaN,12.0,False,True,NaN,ADEGUATO,NaN,COCCHI,NEGATIVO,NaN
2156,27,2016-04-01,2016-03-01,False,False,False,True,False,True,True,...,NaN,12.0,False,False,NaN,ADEGUATO,NaN,COCCHI,NEGATIVO,NaN
2490,55,2016-06-01,NaT,True,False,False,False,False,True,True,...,NaN,12.0,False,False,NaN,NaN,NaN,COCCHI,NEGATIVO,NaN
5682,47,2014-09-01,2014-07-01,False,False,False,True,False,True,True,...,NaN,12.0,False,False,NaN,ADEGUATO,NaN,COCCHI,NEGATIVO,NaN


In [94]:
df.columns

Index(['ETA_PRELIEVO', 'DATA_PRELIEVO', 'DATA_ULTIMA_MESTRUAZIONE',
       'DATA_PAP_PRECEDENTE', 'FORNICE', 'PORTIO', 'CAN_CERV', 'CAV_UT',
       'PERDITE_ER', 'LEUCORREA', 'PRURITO', 'TER_RAD', 'TER_ORM',
       'GRAVIDANZE', 'ABORTI', 'TEST_ESTR', 'TER_ANTIF', 'OP_GIN', 'MENOPAUSA',
       'AMENORREA', 'POST_PARTUM', 'PRIMO_PAP', 'CITO_PREC', 'VIRALE_PREC',
       'BATT_PREC', 'CONTROLLO', 'CTRL_ISTOLOGICO', 'VALIDITA_CAMPIONE',
       'CAUSA_CAMPIONE_NON_VALIDO', 'BATT', 'CITO', 'VIRALE'],
      dtype='object')

In [95]:
df

,ETA_PRELIEVO,DATA_PRELIEVO,DATA_ULTIMA_MESTRUAZIONE,DATA_PAP_PRECEDENTE,FORNICE,PORTIO,CAN_CERV,CAV_UT,PERDITE_ER,LEUCORREA,...,CITO_PREC,VIRALE_PREC,BATT_PREC,CONTROLLO,CTRL_ISTOLOGICO,VALIDITA_CAMPIONE,CAUSA_CAMPIONE_NON_VALIDO,BATT,CITO,VIRALE
4306,54,2018-04-01,NaT,NaT,False,True,True,False,False,False,...,NaN,NaN,NaN,NaN,NaN,ADEGUATO,NaN,NaN,AGC-NOS,NaN
7808,49,2015-11-01,2015-10-01,NaT,False,True,True,False,False,False,...,NaN,NaN,NaN,NaN,NaN,ADEGUATO,NaN,CANDIDA,NEGATIVO,NaN
1487,45,2015-07-01,2015-06-01,NaT,False,True,True,False,False,False,...,NaN,NaN,NaN,NaN,NaN,ADEGUATO,NaN,NaN,ASCUS,NaN
2132,66,2016-04-01,NaT,2012-01-01,False,True,True,False,False,False,...,NEGATIVO,NaN,NaN,NaN,NaN,ADEGUATO,NaN,NaN,AGC-NOS,NaN
2016,41,2016-03-01,2016-02-01,2013-01-01,False,True,True,False,False,False,...,NEGATIVO,NaN,NaN,12.0,NaN,ADEGUATO,NaN,DODERLEIN,NEGATIVO,NaN
2227,24,2016-05-01,2016-04-01,NaT,False,True,True,False,False,False,...,NaN,NaN,NaN,12.0,NaN,ADEGUATO,NaN,COCCHI,NEGATIVO,NaN
2340,29,2016-06-01,2016-05-01,NaT,False,True,True,False,False,True,...,NaN,NaN,NaN,12.0,NaN,ADEGUATO,NaN,COCCHI,NEGATIVO,NaN
2156,27,2016-04-01,2016-03-01,NaT,False,True,True,False,False,False,...,NaN,NaN,NaN,12.0,NaN,ADEGUATO,NaN,COCCHI,NEGATIVO,NaN
2490,55,2016-06-01,NaT,2016-01-01,False,True,True,False,False,False,...,NEGATIVO,HPV,NaN,12.0,NaN,NaN,NaN,COCCHI,NEGATIVO,NaN
5682,47,2014-09-01,2014-07-01,NaT,False,True,True,False,False,False,...,NaN,NaN,NaN,12.0,NaN,ADEGUATO,NaN,COCCHI,NEGATIVO,NaN


In [96]:
df.to_csv('data/report_anonymous_clean.csv', sep=';')